In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.neural_network import MLPClassifier

In [3]:
appstore = pd.read_csv('AppleStore.csv')
appstore = appstore.dropna()

In [4]:
appstore.shape

(7197, 17)

In [5]:
appstore.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


In [6]:
appstore.drop(['Unnamed: 0', 'id', 'track_name'], axis=1, inplace=True)

In [7]:
appstore.describe()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
count,7.197000e+03,7197.000000,7.197000e+03,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000,7197.000000
mean,1.991345e+08,1.726218,1.289291e+04,460.373906,3.526956,3.253578,37.361817,3.707100,5.434903,0.993053
std,3.592069e+08,5.833006,7.573941e+04,3920.455183,1.517948,1.809363,3.737715,1.986005,7.919593,0.083066
min,5.898240e+05,0.000000,0.000000e+00,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000
25%,4.692275e+07,0.000000,2.800000e+01,1.000000,3.500000,2.500000,37.000000,3.000000,1.000000,1.000000
50%,9.715302e+07,0.000000,3.000000e+02,23.000000,4.000000,4.000000,37.000000,5.000000,1.000000,1.000000
75%,1.819249e+08,1.990000,2.793000e+03,140.000000,4.500000,4.500000,38.000000,5.000000,8.000000,1.000000
max,4.025970e+09,299.990000,2.974676e+06,177050.000000,5.000000,5.000000,47.000000,5.000000,75.000000,1.000000


In [9]:
# Print out column names and # of unique values in each categorical variable
appstore.select_dtypes(include=['object']).nunique()

currency          1
ver            1590
cont_rating       4
prime_genre      23
dtype: int64

In [10]:
# Clean up content rating column and convert to numeric
appstore['cont_rating'] = appstore['cont_rating'].str.replace('+', '')
appstore['cont_rating'] = pd.to_numeric(appstore['cont_rating'])

In [11]:
# Drop currency because there is only 1 value,
# and drop ver since it has 1590
appstore.drop(['currency', 'ver'], axis=1, inplace=True)

In [12]:
# Create separate dataframe for numerical values
numerical = appstore.select_dtypes(exclude=['object'])

# Create separate dataframe for categorical values
categorical = pd.get_dummies(appstore.select_dtypes(include='object'))

# Create the combined dataframe
appstore = pd.concat([numerical, categorical], axis=1, sort = False)

In [13]:
Y = appstore['user_rating_ver']

X = appstore

In [14]:
X.head()

,size_bytes,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,cont_rating,sup_devices.num,ipadSc_urls.num,lang.num,...,prime_genre_News,prime_genre_Photo & Video,prime_genre_Productivity,prime_genre_Reference,prime_genre_Shopping,prime_genre_Social Networking,prime_genre_Sports,prime_genre_Travel,prime_genre_Utilities,prime_genre_Weather
0,100788224,3.99,21292,26,4.0,4.5,4,38,5,10,...,0,0,0,0,0,0,0,0,0,0
1,158578688,0.00,161065,26,4.0,3.5,4,37,5,23,...,0,0,1,0,0,0,0,0,0,0
2,100524032,0.00,188583,2822,3.5,4.5,4,37,5,3,...,0,0,0,0,0,0,0,0,0,1
3,128512000,0.00,262241,649,4.0,4.5,12,37,5,9,...,0,0,0,0,1,0,0,0,0,0
4,92774400,0.00,985920,5320,4.5,5.0,4,37,5,45,...,0,0,0,1,0,0,0,0,0,0


In [15]:
# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(10,))
mlp.fit(X, Y)

ValueError: Unknown label type: (array([4.5, 3.5, 4.5, ..., 0. , 4.5, 5. ]),)